## Dram Data Upload

In this assignment, you'll upload Dram shop data to your GBQ account and run a couple of queries against it. The data for this assignment is in a .zip file called `dram-items.zip` on Moodle (though the underlying file will have a date attached to it). These are item-level reports exported from the point-of-sale system at the Dram shop.

Your goals are to upload these to tables in a GBQ data set. Your tables should have the naming convention `dram_items_YYYYMM` and each table should be a single month. 

I recommend extracting the zip file into the repository into a folder called `dram-items`. Make sure _not_ to commit any big data files to your repo.

Make your data pipeline idempotent, which means you will be checking for the presence of your tables and, if they exist, deleting them before recreating them. 


In [1]:
import os
import re
import datetime 

import pandas as pd
import numpy as np
import pandas_gbq
import janitor

# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

### GBQ Set Up

In this next section we connect to our GBQ project and list the data sets inside to test the connection.

In [2]:
# These first two values will be different on your machine. 
service_path = "/Users/chandler/Dropbox/Teaching/"
service_file = 'umt-msba-037daf11ee16.json' # change this to your authentication information  
gbq_proj_id = 'umt-msba' # change this to your project. 

# And this should stay the same. 
private_key =service_path + service_file

In [3]:
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [4]:
# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [5]:
for item in client.list_datasets() : 
    print(item.full_dataset_id)

umt-msba:dram_shop
umt-msba:transactions
umt-msba:wedge_example
umt-msba:wedge_transactions


### Checking for and deleting monthly tables

We'll get all the tables in our Dram data set that match our pattern, then delete them. We do not want to accidentally delete the item lookup table that we put in this data set in class. 

In [6]:
ym_pattern = re.compile(r"20\d{4}")

In [7]:
dataset_id = "dram_shop"

In [8]:
tables = client.list_tables(dataset_id)  

for table in tables:
    
    if ym_pattern.search(table.table_id) : 
        
        full_name = ".".join([gbq_proj_id,dataset_id,table.table_id])
        client.delete_table(full_name, not_found_ok=True)
        print(f"Deleted {full_name}.")
    
    
    

Deleted umt-msba.dram_shop.dram_items_202010.
Deleted umt-msba.dram_shop.dram_items_202011.
Deleted umt-msba.dram_shop.dram_items_202012.


### Reading in and uploading montly tables

In [9]:
# First, a function to make the year-month from dates. 

def reformat_date(date_string) :
    date_string = datetime.datetime.strptime(date_string,"%Y-%m-%d")
    return(datetime.date.strftime(date_string,"%Y%m"))

assert(reformat_date("2022-09-20")=="202209")


In [43]:
item_files = os.listdir("dram-items")
dollar_fields = ['gross_sales','discounts','net_sales','tax']

for file in item_files : 
    item_year = pd.read_csv("dram-items/" + file)
    item_year = janitor.clean_names(item_year)
    
    item_year['ym'] = item_year['date'].map(reformat_date)

    # clean up the dollar signs
    # found this code here: # https://stackoverflow.com/questions/32464280/converting-currency-with-to-numbers-in-python-pandas
    for field in dollar_fields : 
        item_year[field] = item_year[field].\
            apply(lambda x: x.replace('$','')).\
            apply(lambda x: x.replace(',','')).\
            astype(np.float64)
        
    # Some of the years use sku in an unpredictable way, 
    # so I'm just going to overwrite it. 
    item_year['sku'] = ""

    
    unique_yms = item_year['ym'].unique()
    
    uploads = 0
    
    for ym in unique_yms : 
        for_upload = item_year.query(f"ym == '{ym}'")
        for_upload.drop(labels=['ym'],axis="columns",inplace=True)
        
        table_id = f'dram_shop.dram_items_{ym}'
        pandas_gbq.to_gbq(for_upload, table_id, project_id=gbq_proj_id,if_exists="replace")
        uploads += 1
        
    print(f"Just uploaded {uploads} tables from {file}")
    



/Users/chandler/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/chandler/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 8322.03it/s]


Just uploaded 12 tables from items-2020-01-01-2021-01-01.csv


/Users/chandler/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 10645.44it/s]


Just uploaded 12 tables from items-2018-01-01-2019-01-01.csv


100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 12052.60it/s]


...

Just uploaded 12 tables from items-2016-01-01-2017-01-01.csv


100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 8473.34it/s]


Just uploaded 12 tables from items-2021-01-01-2022-01-01.csv


100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 10645.44it/s]

Just uploaded 10 tables from items-2022-01-01-2022-10-01.csv
